In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import torch
import numpy as np
import pandas as pd
import time
import argparse
from functools import partial

# add code directory to path
import sys
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from metrics import top_k_onehot_indicator
from torch_perturb.perturbations import perturbed
from torch_models import NegativeBinomialRegressionModel,NegativeBinomialDebug, torch_bpr_uncurried, deterministic_bpr


2025-01-07 17:16:25.900711: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 17:16:25.942833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 17:16:25.942858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 17:16:25.943863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 17:16:25.951315: I tensorflow/core/platform/cpu_feature_guar

In [2]:

def convert_df_to_3d_array(df):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(df.index, pd.MultiIndex) or set(df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Get unique geoids and timesteps, sorted
    geoids = sorted(df.index.get_level_values('geoid').unique())
    timesteps = sorted(df.index.get_level_values('timestep').unique())

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Initialize the 3D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    num_features = len(df.columns)
    X = np.zeros((num_timesteps, num_locations, num_features))

    # Fill the 3D array
    for (geoid, timestep), row in df.iterrows():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        X[t_idx, g_idx, :] = row.values

    return X, geoids, timesteps

def convert_y_df_to_2d_array(y_df, geoids, timesteps):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(y_df.index, pd.MultiIndex) or set(y_df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Initialize the 2D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    y = np.zeros((num_timesteps, num_locations))

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Fill the 2D array
    for (geoid, timestep), value in y_df.iloc[:, 0].items():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        y[t_idx, g_idx] = value

    return y

def evaluate_model(model, X, y, time, K, M_score_func, perturbed_top_K_func):
    """Evaluate model on given data and return metrics."""
    with torch.no_grad():
        dist = model(X, time)
        
        # Sample and calculate ratio ratings
        y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
        ratio_rating_TMS = y_sample_TMS/y_sample_TMS.sum(dim=-1, keepdim=True)
        ratio_rating_TS = ratio_rating_TMS.mean(dim=1)
        
        # Calculate metrics
        nll = -model.log_likelihood(y, X, time)
        perturbed_bpr_T = torch_bpr_uncurried(ratio_rating_TS, y, K=K, 
                                             perturbed_top_K_func=perturbed_top_K_func)
        deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, y, K=K)
        
        metrics = {
            'nll': nll.item(),
            'perturbed_bpr': torch.mean(perturbed_bpr_T).item(),
            'deterministic_bpr': torch.mean(deterministic_bpr_T).item()
        }
        
        return metrics

def train_epoch_neg_binom(model, optimizer, K, threshold,
                         M_score_func, feat_TSF,
                         time_T, train_y_TS,
                         perturbed_top_K_func, bpr_weight, nll_weight, update=True):
    """Train one epoch of the negative binomial model."""
    model.train()
    optimizer.zero_grad()
    dist = model(feat_TSF, time_T)
    
    y_sample_TMS = dist.sample((M_score_func,)).permute(1, 0, 2)
    y_sample_action_TMS = y_sample_TMS

    ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
    ratio_rating_TS = ratio_rating_TMS.mean(dim=1)
    ratio_rating_TS.requires_grad_(True)

    def get_log_probs_baked(param):
        distribution = model.build_from_single_tensor(param, feat_TSF, time_T)
        log_probs_TMS = distribution.log_prob(y_sample_TMS.permute(1, 0, 2)).permute(1, 0, 2)
        return log_probs_TMS
    
    jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, 
                                                (model.params_to_single_tensor()), 
                                                strategy='forward-mode', 
                                                vectorize=True)

    score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
    score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

    positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), 
                                        K=K, perturbed_top_K_func=perturbed_top_K_func)
    
    if nll_weight > 0:
        bpr_threshold_diff_T = positive_bpr_T - threshold
        violate_threshold_flag = bpr_threshold_diff_T < 0
        negative_bpr_loss = torch.mean(-bpr_threshold_diff_T*violate_threshold_flag)
    else:
        negative_bpr_loss = torch.mean(-positive_bpr_T)
    
    nll = -model.log_likelihood(train_y_TS, feat_TSF, time_T)
    loss = bpr_weight*negative_bpr_loss + nll_weight*nll
    loss.backward()

    loss_grad_TS = ratio_rating_TS.grad
    print(f'Params: {[param for param in model.parameters()]}')
    print(f'Score func estimator: {score_func_estimator_TSP}')
    print(f'Loss grad: {loss_grad_TS}')

    gradient_TSP = score_func_estimator_TSP * torch.unsqueeze(loss_grad_TS, -1)
    gradient_P = torch.sum(gradient_TSP, dim=[0,1])
    gradient_tuple = model.single_tensor_to_params(gradient_P)

    for param, gradient in zip(model.parameters(), gradient_tuple):
        if nll_weight > 0:
            gradient = gradient + param.grad
        param.grad = gradient
        
    if update:
        optimizer.step()

    deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)
    det_bpr = torch.mean(deterministic_bpr_T)

    metrics = {
        'loss': loss.detach().item(),
        'deterministic_bpr': det_bpr.item(),
        'perturbed_bpr': torch.mean(positive_bpr_T).item(),
        'nll': nll.item()
    }

    return metrics, model

def main(K=None, step_size=None, epochs=None, bpr_weight=None,
         nll_weight=None, seed=None, outdir=None, threshold=None,
         perturbed_noise=None, num_score_samples=None, num_pert_samples=None,
         data_dir=None, device='cuda', val_freq=10):
    """Main training loop with command line arguments."""
    
    # Set random seed for reproducibility
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # Load training data
    train_X_df = pd.read_csv(os.path.join(data_dir, 'train_x.csv'), index_col=[0,1])
    train_Y_df = pd.read_csv(os.path.join(data_dir, 'train_y.csv'), index_col=[0,1])
    
    # Load validation data
    val_X_df = pd.read_csv(os.path.join(data_dir, 'valid_x.csv'), index_col=[0,1])
    val_Y_df = pd.read_csv(os.path.join(data_dir, 'valid_y.csv'), index_col=[0,1])
    
    # Process training data
    train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df)#.drop(columns='timestep.1'))
    train_time_arr = np.array([timesteps] * len(geoids)).T
    train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

    # Process validation data
    val_X, _, val_timesteps = convert_df_to_3d_array(val_X_df)#.drop(columns='timestep.1'))
    val_time_arr = np.array([val_timesteps] * len(geoids)).T
    val_y = convert_y_df_to_2d_array(val_Y_df, geoids, val_timesteps)

    # Convert to tensors and move to device
    X_train = torch.tensor(train_X, dtype=torch.float32).to(device)
    y_train = torch.tensor(train_y, dtype=torch.float32).to(device)
    time_train = torch.tensor(train_time_arr, dtype=torch.float32).to(device)
    
    X_val = torch.tensor(val_X, dtype=torch.float32).to(device)
    y_val = torch.tensor(val_y, dtype=torch.float32).to(device)
    time_val = torch.tensor(val_time_arr, dtype=torch.float32).to(device)

    # Initialize model
    model = NegativeBinomialRegressionModel(
        num_locations=len(geoids),
        num_fixed_effects=train_X.shape[2], device=device
    ).to(device)

    # Setup optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

    # Setup top-k function
    top_k_func = partial(top_k_onehot_indicator, k=K)
    perturbed_top_K_func = perturbed(top_k_func, sigma=perturbed_noise, num_samples=num_pert_samples)

    # Initialize metric tracking with separate epoch tracking for validation
    metrics = {
        'train': {
            'epochs': [], 
            'loss': [], 
            'nll': [], 
            'perturbed_bpr': [], 
            'deterministic_bpr': []
        },
        'val': {
            'epochs': [], 
            'nll': [], 
            'perturbed_bpr': [], 
            'deterministic_bpr': []
        },
        'times': []
    }

    best_val_loss = float('inf')
    
    # Training loop
    for epoch in range(epochs):
        print(f'EPOCH: {epoch}')
        start = time.time()
        
        # Training step
        train_metrics, model = train_epoch_neg_binom(
            model, optimizer, K, threshold,
            num_score_samples, X_train, time_train,
            y_train, perturbed_top_K_func,
            bpr_weight, nll_weight, device
        )
        
        # Update training metrics
        metrics['train']['epochs'].append(epoch)
        for metric, value in train_metrics.items():
            metrics['train'][metric].append(value)
        
        # Validation step (every val_freq epochs)
        if epoch % val_freq == 0:
            model.eval()
            val_metrics = evaluate_model(
                model, X_val, y_val, time_val,
                K, num_score_samples, perturbed_top_K_func
            )
            
            # Update validation metrics
            metrics['val']['epochs'].append(epoch)
            for metric, value in val_metrics.items():
                metrics['val'][metric].append(value)
            
            # Save best model
            val_loss = val_metrics['nll'] * nll_weight
            if bpr_weight > 0:
                val_loss -= val_metrics['perturbed_bpr'] * bpr_weight
                
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                if not os.path.exists(outdir):
                    os.makedirs(outdir)
                torch.save(model.state_dict(), f'{outdir}/best_model.pth')
        
        end = time.time()
        metrics['times'].append(end - start)
        
        # Print progress
        print(f"Train - Loss: {train_metrics['loss']:.4f}, NLL: {train_metrics['nll']:.4f}, "
              f"BPR: {train_metrics['deterministic_bpr']:.4f}")
        if epoch % val_freq == 0:
            print(f"Val - NLL: {val_metrics['nll']:.4f}, "
                  f"BPR: {val_metrics['deterministic_bpr']:.4f}")
        
        # Save checkpoints
        if epoch % 100 == 0:
            if not os.path.exists(outdir):
                os.makedirs(outdir)
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'metrics': metrics,
                'best_val_loss': best_val_loss
            }, f'{outdir}/checkpoint.pth')
            
            # Save metrics separately for easier analysis
            # Create DataFrames with proper indexing
            train_df = pd.DataFrame(metrics['train']).set_index('epochs')
            val_df = pd.DataFrame(metrics['val']).set_index('epochs')
            times_df = pd.DataFrame({'times': metrics['times']}, index=range(len(metrics['times'])))
            
            train_df.to_csv(f'{outdir}/train_metrics.csv')
            val_df.to_csv(f'{outdir}/val_metrics.csv')
            times_df.to_csv(f'{outdir}/time_metrics.csv')


In [3]:
#good_nll_model = '/cluster/tufts/hugheslab/kheuto01/opioid_grid_try_fix_params/MA/K100_bw30_nw1_ss0.001_nss100_nps100_seed123_sig0.001_tr0.5'
data_dir = '/cluster/tufts/hugheslab/datasets/NSF_OD/cleaned/MA'

In [4]:
K = 100
bpr_weight = 30
nll_weight = 1
step_size = 0.001
num_score_samples = 100
num_pert_samples = 100
seed = 123
perturbed_noise = 0.001
threshold = 0.5
epochs = 1
outdir = '/cluster/tufts/hugheslab/kheuto01/debug'
data_dir = '/cluster/tufts/hugheslab/datasets/NSF_OD/cleaned/MA'
device = 'cuda'
val_freq = 10

In [5]:
# Load training data
train_X_df = pd.read_csv(os.path.join(data_dir, 'train_x.csv'), index_col=[0,1])
train_Y_df = pd.read_csv(os.path.join(data_dir, 'train_y.csv'), index_col=[0,1])
#temp mem fix
#train_X_df = train_X_df.iloc[:1620]
#train_Y_df = train_Y_df.iloc[:1620]

# Load validation data
val_X_df = pd.read_csv(os.path.join(data_dir, 'valid_x.csv'), index_col=[0,1])
val_Y_df = pd.read_csv(os.path.join(data_dir, 'valid_y.csv'), index_col=[0,1])

# Process training data
train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df)#.drop(columns='timestep.1'))
train_time_arr = np.array([timesteps] * len(geoids)).T
train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
train_X_df.shape

(6480, 14)

In [8]:
# Load model
model_path = os.path.join(good_nll_model, 'best_model.pth')

    
# Initialize model with correct parameters


# Load saved weights
#model.load_state_dict(torch.load(model_path, map_location=device))
#model.eval()

In [8]:
X_train = torch.tensor(train_X, dtype=torch.float32).to(device)
# temp mem fix
#X_train = X_train[:2,:,:]
#timesteps = X_train.shape[0]
train_time_arr = np.array([timesteps] * len(geoids)).T
time_train = torch.tensor(train_time_arr, dtype=torch.float32).to(device)
num_score_samples = 100
num_pert_samples = 100

model = NegativeBinomialDebug(
    num_locations=len(geoids),
    num_fixed_effects=X_train.shape[2], device=device
).to(device)
model=model.to(device)

In [9]:
time_train.shape

torch.Size([4, 1620])

In [10]:
dist = model(X_train, time_train)

y_sample_TMS = dist.sample((num_score_samples,)).permute(1, 0, 2)
y_sample_action_TMS = y_sample_TMS

ratio_rating_TMS = y_sample_action_TMS/y_sample_action_TMS.sum(dim=-1, keepdim=True)
ratio_rating_TS = ratio_rating_TMS.mean(dim=1)
ratio_rating_TS.requires_grad_(True)

def get_log_probs_baked(param):
    distribution = model.build_from_single_tensor(param, X_train, time_train)
    log_probs_TMS = distribution.log_prob(y_sample_TMS.permute(1, 0, 2)).permute(1, 0, 2)
    return log_probs_TMS

jac_TMSP = torch.autograd.functional.jacobian(get_log_probs_baked, 
                                            (model.params_to_single_tensor()), 
                                            strategy='forward-mode', 
                                            vectorize=True)

score_func_estimator_TMSP = jac_TMSP * ratio_rating_TMS.unsqueeze(-1)
score_func_estimator_TSP = score_func_estimator_TMSP.mean(dim=1)    

unconstrained theta: Parameter containing:
tensor([-1.8628], device='cuda:0', requires_grad=True)
Theta: tensor([0.1344], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [12]:
jac_TMSP.min(), jac_TMSP.max()

(tensor(-28.1058, device='cuda:0', grad_fn=<MinBackward1>),
 tensor(35.1680, device='cuda:0', grad_fn=<MaxBackward1>))

In [17]:
y_sample_TMS

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

In [16]:
y_sample_action_TMS.sum(dim=-1, keepdim=True).max()

tensor(0., device='cuda:0')